# Spark with MLRun example

This example notebook demonstrates how to execute a spark job with MLRun.

Our spark job is a generic ETL job which pulls data from user-defined data sources, applies a SQL query on top of them, and writes the result to a user defined destination.

The definition of the input-sources should be according to : https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader

The definition of the output destination should be according to :
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter

In [ ]:
import os
from os.path import isfile, join
from mlrun import new_function, new_task, mlconf

# Set the mlrun database/api
mlconf.dbpath = "http://mlrun-api:8080"

# Set the pyspark script path
V3IO_WORKING_DIR = os.getcwd().replace("/User", "/v3io/" + os.getenv("V3IO_HOME"))
V3IO_SCRIPT_PATH = V3IO_WORKING_DIR + "/spark-function.py"

## Define a task (job parameters)

In [ ]:
# Define a dict of input data sources
DATA_SOURCES = {
    "family": {
        "format": "jdbc",
        "url": "jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam",
        "dbtable": "Rfam.family",
        "user": "rfamro",
        "password": "",
        "driver": "com.mysql.jdbc.Driver",
    },
    "full_region": {
        "format": "jdbc",
        "url": "jdbc:mysql://mysql-rfam-public.ebi.ac.uk:4497/Rfam",
        "dbtable": "Rfam.full_region",
        "user": "rfamro",
        "password": "",
        "driver": "com.mysql.jdbc.Driver",
    },
}

# Define a query to execute on the input data sources
QUERY = "SELECT family.*, full_region.evalue_score from family INNER JOIN full_region ON family.rfam_acc = full_region.rfam_acc  LIMIT 10"

# Define the output destination
WRITE_OPTIONS = {
    "format": "io.iguaz.v3io.spark.sql.kv",
    "mode": "overwrite",
    "key": "rfam_id",
    "path": "v3io://users/admin/frommysql",
}

# Create a task execution with parameters
PARAMS = {"data_sources": DATA_SOURCES, "query": QUERY, "write_options": WRITE_OPTIONS}

SPARK_TASK = new_task(params=PARAMS)

#### Download mysql driver

In [ ]:
!wget https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.19/mysql-connector-java-8.0.19.jar

## Run locally (in the notebook or attched Spark service)

In [ ]:
# Get the list of the dpendency jars
V3IO_JARS_PATH = "/igz/java/libs/"
DEPS_JARS_LIST = [
    join(V3IO_JARS_PATH, f)
    for f in os.listdir(V3IO_JARS_PATH)
    if isfile(join(V3IO_JARS_PATH, f)) and f.startswith("v3io-") and f.endswith(".jar")
]

DEPS_JARS_LIST.append(V3IO_WORKING_DIR + "/mysql-connector-java-8.0.19.jar")

In [ ]:
# Create MLRun function which runs locally in a passthrough mode (since we use spark-submit)
local_spark_fn = new_function(
    kind="local",
    mode="pass",
    command=f"spark-submit --jars {','.join(DEPS_JARS_LIST)} {V3IO_SCRIPT_PATH}",
)

# Run the function with a task
local_spark_fn.run(SPARK_TASK)

## Run as a job on the Kubernetes cluster

In [ ]:
# Create MLRun function to run the spark-job on the kubernetes cluster
serverless_spark_fn = new_function(
    kind="spark", command=V3IO_SCRIPT_PATH, name="my-spark-func"
)

serverless_spark_fn.with_driver_limits(cpu="1300m")
serverless_spark_fn.with_driver_requests(
    cpu=1, mem="4G"
)  # gpu_type & gpus=<number_of_gpus> are supported too
serverless_spark_fn.with_executor_limits(cpu="1400m")
serverless_spark_fn.with_executor_requests(
    cpu=1, mem="4G"
)  # gpu_type & gpus=<number_of_gpus> are supported too

serverless_spark_fn.with_igz_spark()

# Download mysql driver
serverless_spark_fn.spec.build.commands += [
    "wget https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.19/mysql-connector-java-8.0.19.jar "
    "-O /spark/jars/mysql-connector-java-8.0.19.jar"
]

# Set number of executors
serverless_spark_fn.spec.replicas = 2

# Deploy function and install MLRun in the spark image
serverless_spark_fn.deploy()

run = serverless_spark_fn.run(SPARK_TASK, watch=False)

In [ ]:
# Get the spark job's UI URL:
run.ui_url